In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dataset imdb_reviews downloaded and prepared to /Users/catcat/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='/var/folders/sh/lcn4gv4s6tv2br1f7qq88nk40000gn/T/tmpqi_qnzsjtfds',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
       

In [9]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [12]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

#Loop over all training examples and save the sentences and labels
for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

#Loop over all test examples and save the sentences and labels
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [32]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequence = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequence, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [33]:
import tensorflow as tf

In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [37]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [40]:
num_epochs = 10
model.fit(padded, 
         training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 0.6055 - accuracy: 0.6304 - val_loss: 0.3422 - val_accuracy: 0.8500
Epoch 2/10
782/782 [==============================] - 9s 12ms/step - loss: 0.2492 - accuracy: 0.9050 - val_loss: 0.3673 - val_accuracy: 0.8416
Epoch 3/10
782/782 [==============================] - 15s 20ms/step - loss: 0.0969 - accuracy: 0.9770 - val_loss: 0.4514 - val_accuracy: 0.8282
Epoch 4/10
782/782 [==============================] - 18s 24ms/step - loss: 0.0253 - accuracy: 0.9970 - val_loss: 0.5720 - val_accuracy: 0.8162
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0071 - accuracy: 0.9996 - val_loss: 0.5975 - val_accuracy: 0.8265
Epoch 6/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0029 - accuracy: 0.9997 - val_loss: 0.6532 - val_accuracy: 0.8284
Epoch 7/10
782/782 [==============================] - 13s 17ms/step - loss: 9.9838e-04 - accuracy: 1.0000 - val_loss: 0.6990 - val_accuracy

In [42]:
reverse_word_index = tokenizer.index_word

In [47]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + '\n')
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close
out_m.close

<function TextIOWrapper.close()>